In [108]:
import os
import boto3

os.environ["AWS_ENDPOINT_URL"] = "http://localhost/OCF server/S3-Mock-Server/server.php"

# The mock server does not validate AWS_ACCESS_KEY_ID/AWS_SECRET_ACCESS_KEY
os.environ["AWS_ACCESS_KEY_ID"] = "123"
os.environ["AWS_SECRET_ACCESS_KEY"] = "456"

curr_dir = os.getcwd()
file_name = os.path.join(curr_dir, 'client.ipynb')

s3 = boto3.resource('s3',
                    endpoint_url=os.getenv("AWS_ENDPOINT_URL"),
                    verify=True)

## Buckets

### Create Buckets

In [109]:
s3.Bucket("bucket_1").upload_file(file_name, "bucket_1.ipynb")
s3.create_bucket(Bucket='bucket_2')
s3.create_bucket(Bucket='bucket_3')
!tree "$curr_dir/buckets" | tail -n +2

├── bucket_1
│   └── bucket_1.ipynb
├── bucket_2
└── bucket_3

3 directories, 1 file


### List Buckets

In [110]:
for bucket in s3.buckets.all():
    print(bucket.name)
    print(bucket.creation_date)

bucket_1
2022-07-04 00:16:54+08:00
bucket_2
2022-07-04 00:16:54+08:00
bucket_3
2022-07-04 00:16:54+08:00


### Delete Buckets

In [111]:
s3.Bucket("bucket_1").delete() # bucket_1 is not empty
s3.Bucket("bucket_2").delete() # bucket_2 is deleted
s3.Bucket("bucket_3").delete() # bucket_3 is deleted
!tree "$curr_dir/buckets" | tail -n +2

└── bucket_1
    └── bucket_1.ipynb

1 directory, 1 file


## Objects

### Create Objects

In [112]:
bucket = s3.Bucket("bucket_1")
bucket.upload_file(file_name, "1.ipynb")
bucket.upload_file(file_name, "2.ipynb")
bucket.upload_file(file_name, "folder/2.ipynb")
bucket.upload_file(file_name, "folder/3.ipynb")
bucket.upload_file(file_name, "folder/4.ipynb")
bucket.upload_file(file_name, "folder/apple/5.ipynb")
bucket.upload_file(file_name, "folder/application/6.ipynb")
bucket.upload_file(file_name, "folder/apache/7.ipynb")
!tree "$curr_dir/buckets" | tail -n +2

└── bucket_1
    ├── 1.ipynb
    ├── 2.ipynb
    ├── bucket_1.ipynb
    └── folder
        ├── 2.ipynb
        ├── 3.ipynb
        ├── 4.ipynb
        ├── apache
        │   └── 7.ipynb
        ├── apple
        │   └── 5.ipynb
        └── application
            └── 6.ipynb

5 directories, 9 files


### List Objects (prefix)

In [113]:
for obj in s3.Bucket("bucket_1").objects.all():
    print(obj.key)

1.ipynb
2.ipynb
bucket_1.ipynb
folder/2.ipynb
folder/3.ipynb
folder/4.ipynb
folder/apache/7.ipynb
folder/apple/5.ipynb
folder/application/6.ipynb


In [114]:
for obj in s3.Bucket("bucket_1").objects.filter(Prefix="folder/app"):
    print(obj.key)
    print(obj.last_modified)
    print(obj.size)
    obj.delete()
!tree "$curr_dir/buckets" | tail -n +2

folder/apple/5.ipynb
2022-07-04 00:17:07+08:00
6699
folder/application/6.ipynb
2022-07-04 00:17:07+08:00
6699
└── bucket_1
    ├── 1.ipynb
    ├── 2.ipynb
    ├── bucket_1.ipynb
    └── folder
        ├── 2.ipynb
        ├── 3.ipynb
        ├── 4.ipynb
        └── apache
            └── 7.ipynb

3 directories, 7 files


### Download Objects

In [115]:
!ls "$curr_dir"
download_file = os.path.join(curr_dir, "download.ipynb")
s3.Bucket("bucket_1").download_file("folder/apache/7.ipynb", download_file)
!ls "$curr_dir"

LICENSE      buckets      log.txt      temp.xml
README.md    client.ipynb server.php   utils.php
LICENSE        buckets        download.ipynb server.php     utils.php
README.md      client.ipynb   log.txt        temp.xml


In [116]:
os.remove(download_file)
!ls "$curr_dir"

LICENSE      buckets      log.txt      temp.xml
README.md    client.ipynb server.php   utils.php


### Delete Objects

In [117]:
for obj in s3.Bucket("bucket_1").objects.all():
    obj.delete()
!tree "$curr_dir/buckets" | tail -n +2


0 directories, 0 files
